In [8]:
import pytesseract
import cv2
import numpy as np
import platform
if platform.system() == "Windows":
    pytesseract.pytesseract.tesseract_cmd = r'C:\Users\B92383\AppData\Local\Programs\Tesseract-OCR\tesseract.exe'
custom_config = r'--oem 2 --psm 12 -l fra'

# Etape 1
### Standariser l'image

In [9]:

def biggestRectangle(contours):
    max_area = 0
    indexReturn = -1
    for index in range(len(contours)):
        i = contours[index]
        area = cv2.contourArea(i)
        if area > 100:
            peri = cv2.arcLength(i, True)
            approx = cv2.approxPolyDP(i, 0.1 * peri, True)
            if area > max_area:
                max_area = area
                indexReturn = index
    return indexReturn

def process_image(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, None, fx=0.5, fy=0.5, interpolation=cv2.INTER_CUBIC)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    invGamma = 1.0 / 0.3
    table = np.array([((i / 255.0) ** invGamma) * 255 for i in np.arange(0, 256)]).astype("uint8")
    gray = cv2.LUT(gray, table)
    ret, thresh1 = cv2.threshold(gray, 60, 255, cv2.THRESH_BINARY)

    contours, hierarchy = cv2.findContours(thresh1, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)[-2:]

    indexReturn = biggestRectangle(contours)
    hull = cv2.convexHull(contours[indexReturn])

    mask = np.zeros_like(img)
    cv2.drawContours(mask, contours, indexReturn, 255, -1)
    out = np.zeros_like(img)
    out[mask == 255] = img[mask == 255]

    (y, x, _) = np.where(mask == 255)
    (topy, topx) = (np.min(y), np.min(x))
    (bottomy, bottomx) = (np.max(y), np.max(x))
    out = img[topy:bottomy + 1, topx:bottomx + 1, :]

    return out



# Load your image
image_path = "./carte-grise-specimen.jpg"
# image = Image.open(image)
frame = cv2.imread(image_path)
# print(frame)
frame = process_image(frame)



# Etape 2 :
### Trouver les points de chaque element dans la photo

In [10]:
import numpy as np
import cv2

def mouse_drawing(event, x, y, flags, params):
    if event == cv2.EVENT_LBUTTONDOWN:
        print("Left click")
        circles.append((x, y))

circles = []

# Load your image
# image_path = "img\carte-grise-specimen.jpg"
# frame = cv2.imread(image_path)

cv2.namedWindow("Frame")
if len(circles) < 4:
    cv2.setMouseCallback("Frame", mouse_drawing)

while True:
    for center_position in circles:
        cv2.circle(frame, center_position, 2, (0, 0, 255), -1)
    
    points = np.array(circles)
    if len(circles) >= 4:
        cv2.polylines(frame, np.int32([points]), 1, (255, 255, 255))

    cv2.imshow("Frame", frame)

    key = cv2.waitKey(1)
    if key == 27:
        break
    elif key == ord("d"):
        circles = []

cv2.destroyAllWindows()
print(circles)


Left click
[(564, 27)]


In [11]:
regions_dict = {
    'points_N_immatriculation':[(39, 44), (39, 59), (116, 62), (118, 43)],
    'points_titre_document': [(142, 0), (144, 17), (391, 19), (391, 1)],
    'points_d_2': [(47, 303), (48, 319), (387, 318), (386, 303)],
    'points_E': [(343, 344), (343, 361), (496, 365), (493, 343)],
    'points_Date_de_1er_immarticulation': [(173, 45), (174, 61), (268, 61), (266, 44)]
}

# Accessing values for a specific key
print(regions_dict['points_titre_document'][0])

(142, 0)


# Etape 3
### couper les elements et les save 

In [12]:
import cv2

# # Load your image
# image_path = "img\carte-grise-specimen.jpg"
# # image = Image.open(image)
# frame = cv2.imread(image_path)
# # print(frame)
# image = process_image(frame)


# Define a function to trim and save an image
def trim_and_save(region_key, region_points, image):
    x, y, w, h = cv2.boundingRect(np.array(region_points))
    cropped_image = image[y:y+h, x:x+w]
    cv2.imwrite(f"{region_key}_cropped.jpg", cropped_image)

# Iterate through regions and trim/save images
for region_key, region_points in regions_dict.items():
    trim_and_save(region_key, region_points, frame)

# Etape 4
### Passer le PCR pour chaque image

In [13]:
def image2text(cropped_image_path):
    text = ''
    # cropped_image_path = 'points_E_cropped.jpg'
    cropped_image = cv2.imread(cropped_image_path)
    cropped_image = cv2.resize(cropped_image, None,fx=2,fy=2 ,interpolation=cv2.INTER_CUBIC)

    text = pytesseract.image_to_string(cropped_image, config=custom_config)
    # for word in text.split():
    #     if word.__len__() >= 2:
    #         word_found = word
            
            
    
    if len(text)!=0:
        return text
    else:
        print("no text detected")

In [14]:
while True:
    for region_key, value in regions_dict.items():
        cv2.rectangle(frame, value[0], value[2], (0,0,255),2,2)
         # Load the saved cropped image
        cropped_image_path = f"{region_key}_cropped.jpg"
        # cropped_image = cv2.imread(cropped_image_path)
        word_found = image2text(cropped_image_path)
        # print(f"{region_key} : {word_found}")
        cv2.putText(frame, str(word_found), value[0],1,1,(0,0,255),1)

    
 
    cv2.imshow("Frame", frame)

    key = cv2.waitKey(1)
    if key == 27:
        break
  
cv2.destroyAllWindows()